In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline


In [ ]:
# import sys
# sys.path.insert(0, '../')

import numpy as np
import tensorflow as tf
import os, sys, time, pickle, functools
import utils, optimization, metrics, plot, data, evaluation, blocks

import plot
from plot import get_animation

from model import WGanModel, upscale_WGAN_pixel_CNN
from gan import CosmoGAN, UpscaleCosmoGAN
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
fwd = data.fmap.medical_forward
bkd = data.fmap.medical_backward

def non_lin(x):
    return (tf.nn.tanh(x) + 1.0)/2.0
ns = 32
num_sampled = 8

# pathmodel = '../../saved_result'
# pathsample = '/store/nati/archive/medical_code/Samples/medical_16'


pathmodel = '../../saved_result'
# pathmodel = '/store/nati/archive/medical_code/CodeGAN/saved_result'
pathsample = '/store/nati/archive/medical_code/Samples/medical'

# Generate samples - 32->64->256

In [ ]:
name = 'WGAN_0_to_32_4'
# name = 'WGAN_0_to_32'
checkpoint = 36000


params_32 = dict()
params_32['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

resume, params_32 = utils.test_resume(True, params_32)

params_32['cosmology']['backward_map'] = bkd
params_32['cosmology']['forward_map'] = fwd
params_32['summary_dir'] = os.path.join(pathmodel, name +'summary/')
params_32['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

# for ind, item in enumerate(params.items()):
#     print(ind, " : ", item[0], " - ", item[1])
# print(resume)


## 0->32

In [ ]:
wgan_32 = CosmoGAN(params_32, WGanModel, is_3d=True)
gen_samples_32, _ = wgan_32.generate(N=num_sampled, checkpoint=checkpoint)
gen_samples_32 = np.squeeze(gen_samples_32)

In [ ]:
os.makedirs(os.path.join(pathsample,'0_32/'), exist_ok=True)
# utils.save_hdf5(gen_samples_32, os.path.join(pathsample,'0_32/8_32_samples_ckpt_39000.h5'))
utils.save_hdf5(gen_samples_32, os.path.join(pathsample,'0_32/8_32_samples_ckpt_latest.h5'))

## 32->64

In [ ]:
pathmodel = '/store/nati/archive/medical_code/CodeGAN/saved_result'


## Fake 32

In [ ]:
num_samples = 8

# gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'0_32/8_32_samples_ckpt_39000. h5'))[:num_samples]
gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'0_32/8_32_samples_ckpt_latest.h5'))[:num_samples]
print("gen_samples_32 shape=", gen_samples_32.shape)
gen_samples_32 = np.expand_dims(gen_samples_32, axis=4)
print('downsampled shape=', gen_samples_32.shape)

## Real 64

In [ ]:
dataset_64 = data.load.load_medical_dataset(spix=64, scaling=4, augmentation=False, shuffle=False, forward_map=fwd, patch=False)
real_nbody_64 = dataset_64.get_all_data()
print("Nbody dataset shape: ", np.shape(real_nbody_64))

downsampled_32 = blocks.downsample(real_nbody_64[:num_samples], s=2, is_3d=True)
downsampled_32 = np.expand_dims(downsampled_32, axis=4)
print('downsampled_32 shape=', downsampled_32.shape)

In [ ]:
print(np.min(gen_samples_32), np.max(gen_samples_32))
print(np.min(downsampled_32), np.max(downsampled_32))

In [ ]:
# name = 'WGAN_16_to_64'
name = 'WGAN_32_to_64'

checkpoint = None
params_64 = dict()
params_64['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

resume, params_64 = utils.test_resume(True, params_64)

params_64['cosmology']['backward_map'] = bkd
params_64['cosmology']['forward_map'] = fwd
params_64['summary_dir'] = os.path.join(pathmodel, name +'summary/')
params_64['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

In [ ]:
wgan_64 = UpscaleCosmoGAN(params_64, upscale_WGAN_pixel_CNN, is_3d=True)
gen_samples_64 = wgan_64.upscale_image(small=gen_samples_32, checkpoint=None)

In [ ]:
os.makedirs(os.path.join(pathsample,'32_64/'), exist_ok=True)
# utils.save_hdf5(gen_samples_64, os.path.join(pathsample,'32_64/8_64_samples_ckpt_39k_15k.h5'))
utils.save_hdf5(gen_samples_64, os.path.join(pathsample,'32_64/8_64_samples_ckpt_latest.h5'))

## 64->256

## Fake 64

In [ ]:
gen_samples_64 = utils.load_hdf5(os.path.join(pathsample,'32_64/8_64_samples_ckpt_latest.h5'))
gen_samples_64_raw = bkd(gen_samples_64)

print('gen_samples_64 shape=', gen_samples_64.shape)
gen_samples_64 = np.expand_dims(gen_samples_64, axis=4)
print('downsampled shape=', gen_samples_64.shape)

## Real 256

In [ ]:
# real_nbody_256 = data.load.load_medical_dataset(spix=256, scaling=1, augmentation=False, shuffle=False, forward_map=fwd, patch=False).get_all_data()
downsampled_64 = data.load.load_medical_dataset(spix=64, scaling=4, augmentation=False, shuffle=False, forward_map=fwd, patch=False).get_all_data()
downsampled_64_raw = bkd(downsampled_64)
print("downsampled_64 shape: ", np.shape(downsampled_64))

In [ ]:
print(np.min(gen_samples_64), np.max(gen_samples_64))
print(np.min(downsampled_64), np.max(downsampled_64))

In [ ]:
name = 'WGAN_64_to_256'
pathmodel = '/store/nati/archive/medical_code/CodeGAN/saved_result'
params_256 = dict()
params_256['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

resume, params_256 = utils.test_resume(True, params_256)

params_256['cosmology']['backward_map'] = bkd
params_256['cosmology']['forward_map'] = fwd
params_256['summary_dir'] = os.path.join(pathmodel, name +'summary/')
params_256['save_dir'] = os.path.join(pathmodel, name + 'checkpoints/')

In [ ]:
wgan_256 = UpscaleCosmoGAN(params_256, upscale_WGAN_pixel_CNN, is_3d=True)
gen_samples = wgan_256.upscale_image(small=gen_samples_64, checkpoint=None)

In [ ]:
os.makedirs(os.path.join(pathsample,'64_256/'), exist_ok=True)
# utils.save_hdf5(gen_samples, os.path.join(pathsample,'64_256/64_256_samples_ckpt_39k_15k_18k.h5'))
utils.save_hdf5(gen_samples, os.path.join(pathsample,'64_256/64_256_samples_ckpt_latest.h5'))

In [ ]:
# print(np.min(gen_samples), np.max(gen_samples))
# print(np.min(real_nbody_256), np.max(real_nbody_256))